<a href="https://colab.research.google.com/github/Amynechiban/ManhwaAiStudio/blob/main/comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [2]:
#@title Environment Setup


OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = 'ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

Mounting Google Drive...
/
Mounted at /content/drive
/content/drive/MyDrive
/content/drive/MyDrive/ComfyUI
-= Updating ComfyUI =-
Already up to date.
-= Install dependencies =-
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121, https://download.pytorch.org/whl/cu118, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 6.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7

Download some models/checkpoints/vae or custom comfyui nodes (uncomment the commands for the ones you want)

In [3]:
# ==============================================================================
# CELL 2: MODEL MANAGEMENT
# This cell will download your essential models into your permanent ComfyUI
# installation on Google Drive.
# ==============================================================================
import os

# This ensures we are using the same WORKSPACE variable from Cell 1
# If USE_GOOGLE_DRIVE was True, WORKSPACE is "/content/drive/MyDrive/ComfyUI"
# If it was False, WORKSPACE is "/content/ComfyUI"
if 'WORKSPACE' not in locals():
  WORKSPACE = "/content/ComfyUI" # Default if this cell is run alone

# --- Define Paths within the WORKSPACE ---
checkpoint_path = os.path.join(WORKSPACE, "models/checkpoints")
vae_path = os.path.join(WORKSPACE, "models/vae")
lora_path = os.path.join(WORKSPACE, "models/loras")
controlnet_path = os.path.join(WORKSPACE, "models/controlnet")
upscale_path = os.path.join(WORKSPACE, "models/upscale_models")

# --- Create directories if they don't exist ---
for path in [checkpoint_path, vae_path, lora_path, controlnet_path, upscale_path]:
    os.makedirs(path, exist_ok=True)

# --- Download helper function ---
def download_if_not_exists(url, dest_folder):
    filename = url.split('/')[-1]
    dest_path = os.path.join(dest_folder, filename)
    if not os.path.exists(dest_path):
        print(f"Downloading {filename}...")
        os.system(f"wget -c '{url}' -P '{dest_folder}'")
    else:
        print(f"{filename} already exists. Skipping.")

# --- Download all your selected models ---
print("--- Starting Model Downloads ---")
download_if_not_exists("https://huggingface.co/cagliostrolab/animagine-xl-3.1/resolve/main/animagine-xl-3.1.safetensors", checkpoint_path)
download_if_not_exists("https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors", vae_path)
download_if_not_exists("https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_offset_example-lora_1.0.safetensors", lora_path)
download_if_not_exists("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors", controlnet_path)
download_if_not_exists("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors", controlnet_path)
download_if_not_exists("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors", controlnet_path)
download_if_not_exists("https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth", upscale_path)
download_if_not_exists("https://huggingface.co/uwg/real-esrgan-x4-anime-6b/resolve/main/real-esrgan-x4-anime-6b.pth", upscale_path)
print("--- Model Downloads Complete ---")

# --- Install Custom Nodes ---
preprocessor_path = os.path.join(WORKSPACE, "custom_nodes/comfy_controlnet_preprocessors")
if not os.path.exists(preprocessor_path):
    print("Cloning ControlNet Preprocessors...")
    os.system(f"git clone https://github.com/Fannovel16/comfy_controlnet_preprocessors {preprocessor_path}")
    # The install script needs to be run from inside the main ComfyUI directory
    %cd {WORKSPACE}
    os.system(f"pip install -r ./custom_nodes/comfy_controlnet_preprocessors/requirements.txt")
else:
    print("ControlNet Preprocessors already exist.")

--- Starting Model Downloads ---
animagine-xl-3.1.safetensors already exists. Skipping.
sdxl_vae.safetensors already exists. Skipping.
sd_xl_offset_example-lora_1.0.safetensors already exists. Skipping.
control_v11p_sd15_openpose_fp16.safetensors already exists. Skipping.
control_v11p_sd15s2_lineart_anime_fp16.safetensors already exists. Skipping.
control_v11f1p_sd15_depth_fp16.safetensors already exists. Skipping.
RealESRGAN_x4plus.pth already exists. Skipping.
--- Model Downloads Complete ---
Cloning ControlNet Preprocessors...
/content/drive/MyDrive/ComfyUI


### Run ComfyUI with cloudflared (Recommended Way)




In [ ]:
# ==============================================================================
# CELL 3: LAUNCH COMFYUI AND DISPLAY IN NOTEBOOK
# ==============================================================================
import threading
import time
import socket
import os
from google.colab import output

# Use the WORKSPACE variable defined in the first cell
if 'WORKSPACE' not in locals():
  WORKSPACE = "/content/drive/MyDrive/ComfyUI" # Default if running alone, assumes you used Google Drive

%cd {WORKSPACE}

# This function waits for the ComfyUI server to be ready and then displays it
def iframe_thread(port):
  # Wait until the port is open
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()


  print("to open it in a new window you can open this link here:")
  output.serve_kernel_port_as_window(port)

# Start the thread that will display the UI
# We run this in the background so it can wait for the server to start
threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

# Launch ComfyUI
# We removed the --extra-model-paths-config argument because it's no longer needed
# with this setup, as all models are inside the main ComfyUI folder.
!python main.py --dont-print-server

/content/drive/MyDrive/ComfyUI
Checkpoint files will always be loaded safely.
Total VRAM 15095 MB, total RAM 12978 MB
pytorch version: 2.7.0+cu118
xformers version: 0.0.30
Set vram state to: NORMAL_VRAM
Device: cuda:0 Tesla T4 : cudaMallocAsync
Using xformers attention
Python version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
ComfyUI version: 0.3.41
ComfyUI frontend version: 1.22.2
[Prompt Server] web root: /usr/local/lib/python3.11/dist-packages/comfyui_frontend_package/static

Import times for custom nodes:
   0.0 seconds: /content/drive/MyDrive/ComfyUI/custom_nodes/websocket_image_save.py

Context impl SQLiteImpl.
Will assume non-transactional DDL.
No target revision found.
to open it in a new window you can open this link here:
Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>